# Dataset Visualization

### Length of datasets

We first analyze the size of the dataset and respectively of the train, dev, and test set.

In [2]:
import pandas as pd
import numpy as np
from src.models.utils.data import FoodPricingDataset

dataset = FoodPricingDataset(
    img_transform=lambda _: _,
    txt_transform=lambda _: _,
    split=None,
)

print(f"The length of the whole dataset is: {len(dataset)} rows")

The length of the whole dataset is: 7061 rows


In [8]:
df = dataset.data

len_dev = len(df[df["split"] == "dev"])
len_test = len(df[df["split"] == "test"])

def print_len(split: str) -> None:
    len_split = len(df[df["split"] == split])
    print(f"Length of {split} set: {len_split} ({round(100*(len_split) / len(dataset))}%)")


for split in ["train", "dev", "test"]:
    print_len(split)

Length of train set: 4942 (70%)
Length of dev set: 1059 (15%)
Length of test set: 1060 (15%)


### Analysis of the training dataset

We first visualize some examples of images, texts and prices.